Chapter 04: Quantum Fourier Transform and Related Algorithms


Listing 4-1. Quantum Fourier Transform Implementation

In [1]:
!pip install fire --quiet

In [2]:
!pip install elapsedtimer --quiet

In [6]:
import cirq
import numpy as np
import fire
import elapsedtimer
from elapsedtimer import ElapsedTimer
print('cirq version',cirq.__version__)
print('numpy version',np.__version__)
print('fire',fire.__version__)
print('elapsedtimer',elapsedtimer.__version__)


class QFT:
    """
    Quantum Fourier Transform
    Builds the QFT circuit iteratively 
    """

    def __init__(self, signal_length=16,
                 basis_to_transform='',
                 validate_inverse_fourier=False,
                 qubits=None):
        
        self.signal_length = signal_length
        self.basis_to_transform = basis_to_transform
        
        if qubits is None:
            self.num_qubits = int(np.log2(signal_length))
            self.qubits = [cirq.LineQubit(i) for i in range(self.num_qubits)]
        else:
            self.qubits = qubits
            self.num_qubits = len(self.qubits)

        self.qubit_index = 0
        self.input_circuit = cirq.Circuit()

        self.validate_inverse_fourier = validate_inverse_fourier
        self.circuit = cirq.Circuit()
        # if self.validate_inverse_fourier:
        self.inv_circuit = cirq.Circuit()

        for k, q_s in enumerate(self.basis_to_transform):
            if int(q_s) == 1:
                # Change the qubit state from 0 to 1 
                self.input_circuit.append(cirq.X(self.qubits[k]))

    def qft_circuit_iter(self):

        if self.qubit_index > 0:
            # Apply the rotations on the prior qubits
            # conditioned on the current qubit
            for j in range(self.qubit_index):
                diff = self.qubit_index - j + 1
                rotation_to_apply = -2.0 / (2.0 ** diff)
                self.circuit.append(cirq.CZ(self.qubits[self.qubit_index],
                                            self.qubits[j]) ** rotation_to_apply)
        # Apply the Hadamard Transform 
        # on current qubit
        self.circuit.append(cirq.H(self.qubits[self.qubit_index]))
        # set up the processing for next qubit
        self.qubit_index += 1

    def qft_circuit(self):

        while self.qubit_index < self.num_qubits:
            self.qft_circuit_iter()
            # See the progression of the Circuit built
            print(f"Circuit after processing Qubit: {self.qubit_index - 1} ")
            print(self.circuit)
        # Swap the qubits to match qft definititon
        self.swap_qubits()
        print("Circuit after qubit state swap:")
        print(self.circuit)
        # Create the inverse Fourier Transform Circuit
        self.inv_circuit = cirq.inverse(self.circuit.copy())

    def swap_qubits(self):
        for i in range(self.num_qubits // 2):
            self.circuit.append(cirq.SWAP(self.qubits[i], self.qubits[self.num_qubits - i - 1]))

    def simulate_circuit(self):
        sim = cirq.Simulator()
        result = sim.simulate(self.circuit)
        return result


def main(signal_length=16,
         basis_to_transform='0000',
         validate_inverse_fourier=False):

    # Instantiate QFT Class
    _qft_ = QFT(signal_length=signal_length,
                basis_to_transform=basis_to_transform,
                validate_inverse_fourier=validate_inverse_fourier)


    # Build the QFT Circuit
    _qft_.qft_circuit()

    # Create the input Qubit State

    if len(_qft_.input_circuit) > 0:
        _qft_.circuit = _qft_.input_circuit + _qft_.circuit

    if _qft_.validate_inverse_fourier:
        _qft_.circuit += _qft_.inv_circuit

    print("Combined Circuit")
    print(_qft_.circuit)
    # Simulate the circuit

    output_state = _qft_.simulate_circuit()
    # Print the Results
    print(output_state)


if __name__ == '__main__':
    with ElapsedTimer('Execute Quantum Fourier Transform'):
        fire.Fire(main)

cirq version 1.6.1
numpy version 2.3.0
fire 0.7.1
elapsedtimer 1.0.0
Circuit after processing Qubit: 0 
0: ───H───
Circuit after processing Qubit: 1 
0: ───H───@────────────
          │
1: ───────@^-0.5───H───
Circuit after processing Qubit: 2 
                   ┌────────┐
0: ───H───@──────────@───────────────────────
          │          │
1: ───────@^-0.5────H┼──────────@────────────
                     │          │
2: ──────────────────@^-0.25────@^-0.5───H───
                   └────────┘
Circuit after processing Qubit: 3 
                   ┌────────┐   ┌──────────────┐   ┌────────┐
0: ───H───@──────────@─────────────────@─────────────────────────────────────
          │          │                 │
1: ───────@^-0.5────H┼───────────@─────┼─────────────@───────────────────────
                     │           │     │             │
2: ──────────────────@^-0.25─────@^-0.5┼────────────H┼──────────@────────────
                                       │             │          │
3: ────

ERROR: Could not consume arg: --f=c:\Users\user\AppData\Roaming\jupyter\runtime\kernel-v3b453c2cf66f5cc45dbfca98b7b4c666d5cf2dbd2.json
Usage: ipykernel_launcher.py -

For detailed information on this command, run:
  ipykernel_launcher.py - --help


25.341 ms: Execute Quantum Fourier Transform


FireExit: 2

C:\Users\user\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Listing 4-2. Quantum Phase Estimation

In [1]:
import cirq
print('cirq version', cirq.__version__)
from quantum_fourier_transform import QFT

class quantum_phase_estimation:
    
    def __init__(self, num_input_state_qubits=3,
                 num_ancillia_qubits=5,
                 unitary_transform=None,
                 U=None,
                 input_state=None):

        
        self.num_ancillia_qubits = num_ancillia_qubits
        
        self.output_qubits = [cirq.LineQubit(i) for i in range(self.num_ancillia_qubits)]
        
        self.input_circuit = cirq.Circuit()
        self.input_state =  input_state
        
        if self.input_state is not None:
            self.num_input_qubits = len(self.input_state)
        else:
            self.num_input_qubits = num_input_state_qubits
            
        self.input_qubits = [cirq.LineQubit(i) for i in
                             range(self.num_ancillia_qubits,
                                   self.num_ancillia_qubits + num_input_state_qubits)]

        if self.input_state is not None:
            for i, c in enumerate(self.input_state):
                if int(c) == 1:
                    self.input_circuit.append(cirq.X(self.input_qubits[i]))
                    
        self.unitary_transform = unitary_transform
        if self.unitary_transform is None:
            self.U = cirq.I
        elif self.unitary_transform == 'custom':
            self.U = U
        elif self.unitary_transform == 'Z':
            self.U = cirq.CZ
        elif self.unitary_transform == 'X':
            self.U = cirq.CX
        else:
            raise NotImplementedError(f"self.unitary transform not Implemented")

        self.circuit = cirq.Circuit()
        
        
    def phase_1_create_circuit_iter(self):
        
        for i in range(self.num_ancillia_qubits):
            self.circuit.append(cirq.H(self.output_qubits[i]))
            _pow_ = 2**(self.num_ancillia_qubits - 1 - i)
            #_pow_ = 2 ** (i)
            for k in range(self.num_input_qubits):
                print(self.U)
                self.circuit.append(self.U(self.output_qubits[i], self.input_qubits[k])**_pow_)
        
        
    def inv_qft(self):
        self._qft_ = QFT(qubits=self.output_qubits)
        self._qft_.qft_circuit()


    def simulate_circuit(self,circ):
        sim = cirq.Simulator()
        result = sim.simulate(circ)
        return result   
    
def main(num_input_state_qubits=1,
                 num_ancillia_qubits=2,
                 unitary_transform='Z',
                 U=None,input_state='1'):
    
    _QP_ = quantum_phase_estimation(num_ancillia_qubits=num_ancillia_qubits,
                                    num_input_state_qubits=num_input_state_qubits,
                                    unitary_transform=unitary_transform,
                                    input_state=input_state)
    _QP_.phase_1_create_circuit_iter()
    
    _QP_.inv_qft()
    
    circuit = _QP_.circuit  + _QP_._qft_.inv_circuit
    if len(_QP_.input_circuit) > 0:
        circuit = _QP_.input_circuit + circuit

    print(circuit)
    result = _QP_.simulate_circuit(circuit)
    print(result)

if __name__ == '__main__':
    main()

cirq version 1.6.1
cirq version 1.6.1
numpy version 2.3.0
fire 0.7.1
elapsedtimer 1.0.0
CZ
CZ
Circuit after processing Qubit: 0 
0: ───H───
Circuit after processing Qubit: 1 
0: ───H───@────────────
          │
1: ───────@^-0.5───H───
Circuit after qubit state swap:
0: ───H───@────────────×───
          │            │
1: ───────@^-0.5───H───×───
0: ───────H───@─────────×───────@───────H───
              │         │       │
1: ───────H───┼─────@───×───H───@^0.5───────
              │     │
2: ───X───────@^0───@───────────────────────
measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2))
output vector: |101⟩

phase:
output vector: |⟩


Listing 4-3. Period Finding Implementation

In [2]:
import cirq
from quantum_fourier_transform import QFT
import numpy as np
from fractions import Fraction
print('cirq version',cirq.__version__)
print('numpy version',np.__version__)

def euclid_gcd(a, b):
    if b == 0:
        return a
    else:
        return euclid_gcd(b, a % b)


"""
The Period Finding Class computes the Period of functions
of the form f(x) = a^x mod N using Quantum Phase Estimation
Alternately we can say the algorithm finds the period of 
the element a mod N
"""


class PeriodFinding:

    def __init__(self,
                 ancillia_precision_bits=4,
                 func_domain_size=16,
                 a=7,
                 N=15
                 ):

        self.ancillia_precision_bits = ancillia_precision_bits
        self.func_domain_size = func_domain_size
        self.num_output_qubits = self.ancillia_precision_bits
        self.num_input_qubits = int(np.log2(self.func_domain_size))
        self.output_qubits = [cirq.LineQubit(i) for i in range(self.num_output_qubits)]
        self.input_qubits = [cirq.LineQubit(i) for i in range(self.num_output_qubits,
                                                              self.num_output_qubits + self.num_input_qubits)]
        self.a = a

        self.N = N
        if self.N is None:
            self.N = func_domain_size - 1

        self.circuit = cirq.Circuit()

    def periodic_oracle(self, a, m, k):
        """
        Implement an oracle U_a that takes in the state  
        input state |y> and outputs |ay mod N>
        """

        for i in range(m):
            if a in [2, 13]:
                self.circuit.append(cirq.SWAP(self.input_qubits[0],
                                              self.input_qubits[1]).controlled_by(self.output_qubits[k]))
                self.circuit.append(cirq.SWAP(self.input_qubits[1],
                                              self.input_qubits[2]).controlled_by(self.output_qubits[k]))
                self.circuit.append(cirq.SWAP(self.input_qubits[2],
                                              self.input_qubits[3]).controlled_by(self.output_qubits[k]))

            if a in [7, 8]:
                self.circuit.append(cirq.SWAP(self.input_qubits[2],
                                              self.input_qubits[3]).controlled_by(self.output_qubits[k]))
                self.circuit.append(cirq.SWAP(self.input_qubits[1],
                                              self.input_qubits[2]).controlled_by(self.output_qubits[k]))
                self.circuit.append(cirq.SWAP(self.input_qubits[0],
                                              self.input_qubits[1]).controlled_by(self.output_qubits[k]))
            if a in [4, 11]:
                self.circuit.append(cirq.SWAP(self.input_qubits[1],
                                              self.input_qubits[3]).controlled_by(self.output_qubits[k]))
                self.circuit.append(cirq.SWAP(self.input_qubits[0],
                                              self.input_qubits[2]).controlled_by(self.output_qubits[k]))

            if a in [7, 11, 13]:
                for j in range(self.num_input_qubits):
                    self.circuit.append(cirq.X(self.input_qubits[j]).controlled_by(self.output_qubits[k]))

    def build_phase_1_period_finding_circuit(self):

        # Apply Hadamard Transform on each output qubit

        self.circuit.append([cirq.H(self.output_qubits[i]) for i in range(self.num_output_qubits)])
        # Set input qubit to state |0001>
        self.circuit.append(cirq.X(self.input_qubits[-1]))

        if euclid_gcd(self.N, self.a) != 1:
            print(f"{self.a} is not co-prime to {self.N}")
            co_primes = []
            for elem in range(2, self.N):
                if euclid_gcd(self.N, elem) == 1:
                    co_primes.append(elem)
            print(f"Select a from the list of co-primes to {self.N}: {co_primes} ")

        else:
            print(f"Trying period finding of element a = {self.a} mod {self.N}")
            a = self.a

        for q in range(self.num_output_qubits):
            _pow_ = 2 ** (self.num_output_qubits - q - 1)
            self.periodic_oracle(a=a, m=_pow_, k=q)

    def inv_qft(self):
        """
        Inverse Fourier Transform
        :return: 
        IFT circuit
        """
        self._qft_ = QFT(qubits=self.output_qubits)
        self._qft_.qft_circuit()

    def simulate_circuit(self, circ):
        """
        Simulates the Period Finding Algorithm 
        :param circ: Circuit to Simulate 
        :return: Output results of Simulation 
        """
        circ.append([cirq.measure(*self.output_qubits, key='Z')])
        sim = cirq.Simulator()
        result = sim.run(circ, repetitions=1000)
        out = dict(result.histogram(key='Z'))
        out_result = {}
        for k in out.keys():
            new_key = "{0:b}".format(k)
            if len(new_key) < self.num_output_qubits:
                new_key = (self.num_output_qubits - len(new_key)) * '0' + new_key
            # print(new_key,k)
            out_result[new_key] = out[k]

        return out_result

    def measurement_to_period(self, results, denom_lim=15):
        # Convert a state to Phase as a binary fraction 
        # |x_1,x_2....x_n> -> x_1*2^-1 + x_2*2^-2 + .. + x_n*2^-n  
        measured_states = list(results.keys())

        measured_phase = []
        measured_phase_rational = []

        for s in measured_states:
            phase = int(s, 2) / (2 ** len(s))
            phase_rational = Fraction(phase).limit_denominator(denom_lim)
            measured_phase.append(phase)
            measured_phase_rational.append(phase_rational)
        print(f"---------------------------------")
        print(f"Measured  |   Real   |   Rational")
        print(f"State     |   Phase  |    Phase  ")
        print(f"---------------------------------")
        for i in range(len(measured_phase)):
            print(f"    {measured_states[i]}  |  {measured_phase[i]}    |  {measured_phase_rational[i]}")
            print(f"---------------------------------")
        print('\n')

        max_phase_rational = measured_phase_rational[np.argmax(np.array(measured_phase))]
        max_phase_numerator = max_phase_rational.numerator
        max_phase_denominator = max_phase_rational.denominator
        if (max_phase_denominator - max_phase_numerator) == 1:
            period = max_phase_denominator
        else:
            print(f"period cannot be determined")
            period = np.inf

        return period


def period_finding_routine(func_domain_size=16,
         ancillia_precision_bits=4,
         a=7,
         N=15):
    """

    :param func_domain_size:
        States in the Domain of the function.
    :param ancillia_precision_bits:
        Precision bits for Phase Measurement
    :param N: Generally func_domain_size - 1 
    :param a:  Element whose periodicity mod N 
               is to be computed
    :return: Period r of the element a mod N 
     """

    _PF_ = PeriodFinding(ancillia_precision_bits=ancillia_precision_bits,
                         func_domain_size=func_domain_size,
                         a=a,
                         N=N)
    _PF_.build_phase_1_period_finding_circuit()

    _PF_.inv_qft()

    circuit = _PF_.circuit + _PF_._qft_.inv_circuit

    print(circuit)
    result = _PF_.simulate_circuit(circuit)
    print(f"Measurement Histogram Results follow")
    print(result)
    print('\n')
    period = _PF_.measurement_to_period(result, denom_lim=_PF_.N)
    print(f"Period of {a} mod {N} is: {period} ")
    return period


if __name__ == '__main__':
    period_finding_routine()

cirq version 1.6.1
numpy version 2.3.0
Trying period finding of element a = 7 mod 15
Circuit after processing Qubit: 0 
0: ───H───
Circuit after processing Qubit: 1 
0: ───H───@────────────
          │
1: ───────@^-0.5───H───
Circuit after processing Qubit: 2 
                   ┌────────┐
0: ───H───@──────────@───────────────────────
          │          │
1: ───────@^-0.5────H┼──────────@────────────
                     │          │
2: ──────────────────@^-0.25────@^-0.5───H───
                   └────────┘
Circuit after processing Qubit: 3 
                   ┌────────┐   ┌──────────────┐   ┌────────┐
0: ───H───@──────────@─────────────────@─────────────────────────────────────
          │          │                 │
1: ───────@^-0.5────H┼───────────@─────┼─────────────@───────────────────────
                     │           │     │             │
2: ──────────────────@^-0.25─────@^-0.5┼────────────H┼──────────@────────────
                                       │             │   

Listing 4-4. Factoring Implementation

In [ ]:
!pip install periodfinding

ERROR: Could not find a version that satisfies the requirement period-finding (from versions: none)
ERROR: No matching distribution found for period-finding


In [5]:
import cirq
from period_finding import period_finding_routine
from period_finding import euclid_gcd
import numpy as np
print('numpy version',np.__version__)


class Factoring:
    """"
    Find the factorization of number N = p*q
    where p and q are prime to each other  
    """

    def __init__(self, N):
        self.N = N

    def factoring(self):

        prev_trials_for_a = []
        factored = False

        while not factored:
            new_a_found = False

            # Sample a new "a" not already sampled 
            while not new_a_found:
                a = np.random.randint(2, self.N)
                if a not in prev_trials_for_a:
                    new_a_found = True

            # "a" not co-prime to N are not periodic 
            if euclid_gcd(self.N, a) == 1:
                # Call the period_finding_routine from PeriodFinding 
                # Implementation
                period = period_finding_routine(a=a, N=self.N)

                # Check if the period is even.
                # It period even (a^(r/2))^2 = 1 mod (N) 
                # for integer a^(r/2)
                if period % 2 == 0:

                    # Check if a^(r/2) != +/- 1 mod(N)
                    # if condition satisfied number gets 
                    # factorized in this iteration
                    if a ** (period / 2) % self.N not in [+1, -1]:
                        prime_1 = euclid_gcd(self.N, a ** (period / 2) + 1)
                        prime_2 = euclid_gcd(self.N, a ** (period / 2) - 1)
                        factored = True
                        return prime_1, prime_2
            else:
                # If we have exhausted all "a"s and
                # still havent got prime factors something 
                # is off
                if len(prev_trials_for_a) == self.N - 2:
                    raise ValueError(f"Check input is a product of two primes")


if __name__ == '__main__':

    fp = Factoring(N=15)
    factor_1, factor_2 = fp.factoring()

    if factor_1 is not None:
        print(f"The factors of {fp.N} are {factor_1} and {factor_2}")
    else:
        print(f"Error in factoring")

cirq version 1.6.1
numpy version 2.3.0
numpy version 2.3.0
Trying period finding of element a = 2 mod 15
Circuit after processing Qubit: 0 
0: ───H───
Circuit after processing Qubit: 1 
0: ───H───@────────────
          │
1: ───────@^-0.5───H───
Circuit after processing Qubit: 2 
                   ┌────────┐
0: ───H───@──────────@───────────────────────
          │          │
1: ───────@^-0.5────H┼──────────@────────────
                     │          │
2: ──────────────────@^-0.25────@^-0.5───H───
                   └────────┘
Circuit after processing Qubit: 3 
                   ┌────────┐   ┌──────────────┐   ┌────────┐
0: ───H───@──────────@─────────────────@─────────────────────────────────────
          │          │                 │
1: ───────@^-0.5────H┼───────────@─────┼─────────────@───────────────────────
                     │           │     │             │
2: ──────────────────@^-0.25─────@^-0.5┼────────────H┼──────────@────────────
                                     